In [ ]:
%load_ext autoreload
%autoreload 2
from symbolic_bounds import DAG

# Create a DAG
dag = DAG()

# Add binary nodes - all nodes must be assigned to a partition
X = dag.add_node('X', support={0, 1}, partition='L')
Y = dag.add_node('Y', support={0, 1}, partition='R')
dag.add_edge(X, Y)

# Enumerate all response types for Y given parent X
# This will create 2^2 = 4 response types:
# RT1: {((X,0),)->0, ((X,1),)->0}  (always 0)
# RT2: {((X,0),)->0, ((X,1),)->1}  (copy X)
# RT3: {((X,0),)->1, ((X,1),)->0}  (negate X)
# RT4: {((X,0),)->1, ((X,1),)->1}  (always 1)
response_types = dag.enumerate_response_types(Y)

# Print the response type table
dag.print_response_type_table(Y)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Response types for Y (parents: X):
=====

r_Y^1:
X | Y
-----
0 | 0
1 | 0

r_Y^2:
X | Y
-----
0 | 0
1 | 1

r_Y^3:
X | Y
-----
0 | 1
1 | 0

r_Y^4:
X | Y
-----
0 | 1
1 | 1



In [ ]:
# Create a more complex DAG: Z -> X -> Y with additional edge Z -> Y
dag2 = DAG()

# Add binary nodes - all must be assigned to a partition
Z = dag2.add_node('Z', support={0, 1}, partition='L')
X = dag2.add_node('X', support={0, 1}, partition='R')
Y = dag2.add_node('Y', support={0, 1}, partition='R')

# Add edges: Z -> X -> Y and Z -> Y
dag2.add_edge(Z, X)
dag2.add_edge(X, Y)
dag2.add_edge(Z, Y)


print("DAG structure:", dag2)
print(f"\nW_L: {{{', '.join(n.name for n in dag2.W_L)}}}")
print(f"W_R: {{{', '.join(n.name for n in dag2.W_R)}}}")

# Enumerate response types for all nodes
print("\n" + "="*60)
print("RESPONSE TYPES FOR ALL NODES")
print("="*60)

# Z has no parents
dag2.enumerate_response_types(Z)
dag2.print_response_type_table(Z)

# X has parent Z
dag2.enumerate_response_types(X)
dag2.print_response_type_table(X)

# Y has parents Z and X
dag2.enumerate_response_types(Y)
dag2.print_response_type_table(Y)

DAG structure: DAG(nodes=[Y, X, Z], edges=[Z->X, Z->Y, X->Y])

W_L: {Z}
W_R: {Y, X}

RESPONSE TYPES FOR ALL NODES

Response types for Z (no parents):
r_Z^1: Z = 0
r_Z^2: Z = 1


Response types for X (parents: Z):
=====

r_X^1:
Z | X
-----
0 | 0
1 | 0

r_X^2:
Z | X
-----
0 | 0
1 | 1

r_X^3:
Z | X
-----
0 | 1
1 | 0

r_X^4:
Z | X
-----
0 | 1
1 | 1


Response types for Y (parents: X, Z):

r_Y^1:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 0
1 | 0 | 0
1 | 1 | 0

r_Y^2:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 0
1 | 0 | 0
1 | 1 | 1

r_Y^3:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 0
1 | 0 | 1
1 | 1 | 0

r_Y^4:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 0
1 | 0 | 1
1 | 1 | 1

r_Y^5:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 0
1 | 1 | 0

r_Y^6:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 0
1 | 1 | 1

r_Y^7:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 0

r_Y^8:
X | Z | Y
---------
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 1

r_Y^9:
X | Z | Y
---------
0 | 0 | 1
0 | 1 | 0
1 | 0 | 0
1 | 1 | 0



## More Complex Example: Z -> X -> Y with Z in W_L